In [15]:
import requests
import csv
import json
from datetime import datetime
from collections import defaultdict

In [16]:
url = "https://hub.snapshot.org/graphql?"
headers = {
    "accept": "application/json",
    "accept-language": "vi,en-US;q=0.9,en;q=0.8",
    "content-type": "application/json",
}

proposals = []

In [ ]:
from datetime import timedelta, timezone

query = """
query Proposals {
  proposals(
    first: 1000,
    skip: 0,
    orderBy: "created",
    orderDirection: asc, where: {space: "uniswapgovernance.eth"}) {
    id
    created
  }
}
"""

# Prepare the request payload
payload = {
    "query": query,
    "variables": None,
    "operationName": "Proposals",
}

response = requests.post(url, headers=headers, json=payload)

print(response.json())

if response.status_code == 200:
    data = response.json()
    fetched_proposals = data.get("data", {}).get("proposals", [])
    daily_counts = defaultdict(int)
    for proposal in fetched_proposals:
        date = datetime.fromtimestamp(proposal["created"], tz=timezone.utc).strftime('%Y-%m-%d')
        daily_counts[date] += 1

    dates = [datetime.fromtimestamp(proposal["created"], tz=timezone.utc).date() for proposal in fetched_proposals]
    start_date = min(dates)
    end_date = max(dates)

    full_time_series = []
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime('%Y-%m-%d')
        count = daily_counts.get(date_str, 0)
        full_time_series.append((date_str, count))
        current_date += timedelta(days=1)

    output_file = "mantle_proposals_time_series.csv"
    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Date", "Proposal Count"])  # Write header
        writer.writerows(full_time_series)
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")